In [1]:
from hog_classifier import HogClassifier, load_model
from config import Config
from pdf_utils import * 
from extractor import SfInfoExtractor

import argparse
import warnings
warnings.filterwarnings('ignore')


config = Config('config.yaml')
extractor = SfInfoExtractor(config)
orient_clf = load_model('models/orient.pkl')
type_clf = load_model('models/type.pkl')



In [2]:
import pandas as pd
df = pd.read_csv('output/csv/ocr_dpi.csv', dtype={'sf_no': str,'buyer_inn': str,'buyer_kpp': str,'seller_inn':str,'seller_kpp':str})

In [3]:
tdf = df[~df.sf_no.isna() | ~df.sf_date.isna() | ~df.buyer_inn.isna() | ~df.buyer_kpp.isna() | ~df.seller_inn.isna() | ~df.seller_kpp.isna()]
tdf = tdf[tdf.sf_no.isna() | tdf.sf_date.isna() | tdf.buyer_inn.isna() | tdf.buyer_kpp.isna() | tdf.seller_inn.isna() | tdf.seller_kpp.isna()] 

In [ ]:
print(tdf.sf_no.isna().sum())
tdf[tdf.sf_no.isna()].file_name

In [15]:
import cv2
import os

data = []
for file_name in tdf[tdf.sf_no.isna()].file_name.values[:10]:
    file_name = os.path.join('dataset/png', file_name)    
    img = cv2.imread(file_name)
    h,w = img.shape[:2]
    d = dpi(w,h)
    # заголовок страницы с инфо по с/ф
    head_img = extractor.head_roi(img)
    text = ocr_rus(head_img, config = f'--oem 1 --psm 6')
    # text = ocr_rus(head_img, config = f'--oem 1 --psm 12 --dpi {d}')
    info = extractor.extract_sf_data(text)
    text = [ x for x in text.split('\n') if len(x) > 2]    
    data.append(info)
    print(file_name,h,w,d)
    print(info)
    print('\n'.join(text))
    # break

l_sf_no = []
for x in data:
    l_sf_no.append(x['sf_no'])
l_sf_no

# ['1247', '259', '204НЕРО0', 'ПЕРЕДАТОЧНЫЙДОКУМЕНТ2', '0311007КУ', '', 'ЦО4796', None, '108','87'] 3
# ['1247', '', None, 'ПЕРЕДАТОЧНЫЙДОКУМЕНТ2', '0311007КУ', '', 'ЦО4796', None, '108', '87'] 3 d
# ['1247', '259', None, '89', '0З11007КУ', 'ПОНИ3', 'ЦО4796', None, '108', '87'] 4
# ['1247', '1259', None, None, '0З11007КУ', '888', 'ЦО4796', None, '108', '87'] 4 d
# ['1247', '1259', '', '9.ЭФЕС0АООАДААРЕННОСТАИННЕПРИОНЛЬСТАРООСНКНОЙФЕДЕРАЦИЯСТ19МАГУ2017Г.1991', '0311007КУ', '888', 'ЦО4796', None, '108', '87'] 6
# ['1247', '1259', '', '9.ЭФЕС0АООАДААРЕННОСТАИННЕПРИОНЛЬСТАРООСНКНОЙФЕДЕРАЦИЯСТ19МАГУ2017Г.1991', '0311007КУ', '888', 'ЦО4796', None, '108','87'] 6 d
# [None, '1259', None, '', '', '888', '', None, '108', ''] 11
# [None, '', '', '', '', '', '', None, '', ''] 11 d
# [None, '1259', None, '', '', '888', '', None, '108', ''] 12
# [None, '', '', '', '', '', '', None, '', ''] 12 d



2021-11-16 12:36:52.881 | DEBUG    | extractor:head_roi:287 - table roi (181, 646, 3223, 304)
2021-11-16 12:36:53.767 | DEBUG    | extractor:sf_no_and_date:93 - Entering 'extract_sf_no' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, ' 1247 ОТ '), kwargs={})
2021-11-16 12:36:53.768 | DEBUG    | extractor:sf_no_and_date:93 - Exiting 'extract_sf_no' (result=1247)
2021-11-16 12:36:53.768 | DEBUG    | extractor:sf_no_and_date:94 - Entering 'extract_date' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, '31 МАЯ 2021'), kwargs={})
2021-11-16 12:36:53.769 | DEBUG    | extractor:sf_no_and_date:94 - Exiting 'extract_date' (result=31.05.2021)


dataset/png/MEJ4uVNBE9zKQMmIEzy9sbcnkOyTx8v5bDCyeGyk_2_sf_0.png 2480 3507 300
{'sf_no': '1247', 'sf_date': '31.05.2021', 'buyer_inn': '5607143633', 'buyer_kpp': '560701001', 'seller_inn': None, 'seller_kpp': None}
., Приложение № 1 « постановлению Правительства Российской Федерации от со декаюря сылка нем 1 лэй
О ‚ва (в редакции постановления Правительства Российской Федерации эт 19 авсуста 2017 г № 981)
* Счет-фактура № 1247 от 31 мая 2021 г.
Испоавление № -- от --
Продавец: АО "ПОЛЕМА"
Адрес: 300016. Тульская обл, Тула г. Пржевальского ул. дом № 3
ИНН/КПП продавца: 7105008070/7 10501001
Фрузоотправитель и его адрес: он же
Грузополучатель и его адрес: Общество с ограниченной ответственностью "ТРАЙРИМ МЕТАЛЗ".462359. Оренбургская обл., Новотроицк г.. Ленина пл. дом № 4, квартира 23
К платежно-расчетному документу № от
Покупатель: Общество с ограниченной ответственностью "ТРАЙРИМ МЕТАЛЗ"
Адрес: 462359. Оренбургская обл.: Новотроицк г.. Ленина пл, дом № 4. квартира 23
ИНН/КПП покупателя:

2021-11-16 12:36:54.021 | DEBUG    | extractor:head_roi:287 - table roi (138, 572, 3232, 409)
2021-11-16 12:36:54.878 | DEBUG    | extractor:sf_no_and_date:93 - Entering 'extract_sf_no' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, ' 1259 ОТ '), kwargs={})
2021-11-16 12:36:54.879 | DEBUG    | extractor:sf_no_and_date:93 - Exiting 'extract_sf_no' (result=1259)
2021-11-16 12:36:54.879 | DEBUG    | extractor:sf_no_and_date:94 - Entering 'extract_date' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, '05 ИЮЛЯ 2021'), kwargs={})
2021-11-16 12:36:54.880 | DEBUG    | extractor:sf_no_and_date:94 - Exiting 'extract_date' (result=05.07.2021)
2021-11-16 12:36:54.984 | DEBUG    | extractor:head_roi:287 - table roi (92, 479, 2133, 468)


dataset/png/eWwqyMBOt2qLdQ4MhyaIjktVGpCwwPIE4hUunYgr_5_sf_0.png 2549 3509 304
{'sf_no': '1259', 'sf_date': '05.07.2021', 'buyer_inn': '1661008339', 'buyer_kpp': '166101001', 'seller_inn': '1658008515', 'seller_kpp': '165801001'}
Счет-фактура № 1259 от 05 июля 2021 (0) Дааа нон оееонлены нае узавнедре сан: щей: чрное ый сре А |
Исправление № -- от -- (1а)
Продавец: АО "СТАРТ" (2) Покупатель: АО "Казанское ОКБ "Союз" (6)
Адрес: Адрес: 420095, Татарстан Респ, Казань г, Восстания ул, дом № 100, корпус 315 (а) Адрес: 420036, г.Казань, ул.Дементьева, 1 (ба)
ИНН/КПП продавца: 1658008515/165801001 (26) ИНН/КПП покупателя: 1661008339/166101001 (66)
Грузоотправитель и его адрес: он же (3) Валюта: наименование, код Российский рубль, 643 (7)
Грузополучатель и его адрес: АО "Казанское ОКБ "Союз",420036, г.Казань, ул.Дементьева, 1 (4) Идентификатор государственного контракта,
К платежно-расчетному документу № 5785 от 02.07.2021 г. (5) договора (соглашения) (при наличии): ИГК 202618730996141220801742

2021-11-16 12:36:55.972 | DEBUG    | extractor:sf_no_and_date:98 - Entering 'extract_sf_no' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, 'И РОВНЕЕ РЕА Е  ОСУ ОВЕН НН ТЕННЕННЕ НЕЕЕ НЕЕНННННСОС Ц'), kwargs={})
2021-11-16 12:36:55.973 | DEBUG    | extractor:sf_no_and_date:98 - Exiting 'extract_sf_no' (result=)
2021-11-16 12:36:55.974 | DEBUG    | extractor:sf_no_and_date:100 - Entering 'extract_date' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, ''), kwargs={})
2021-11-16 12:36:55.974 | DEBUG    | extractor:sf_no_and_date:100 - Exiting 'extract_date' (result=None)


...

2021-11-16 12:37:02.056 | DEBUG    | extractor:head_roi:287 - table roi (14, 747, 3379, 806)
2021-11-16 12:37:03.655 | DEBUG    | extractor:sf_no_and_date:93 - Entering 'extract_sf_no' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, ' 108 ОТ '), kwargs={})
2021-11-16 12:37:03.656 | DEBUG    | extractor:sf_no_and_date:93 - Exiting 'extract_sf_no' (result=108)
2021-11-16 12:37:03.656 | DEBUG    | extractor:sf_no_and_date:94 - Entering 'extract_date' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, '10 МАРТА 2021'), kwargs={})
2021-11-16 12:37:03.657 | DEBUG    | extractor:sf_no_and_date:94 - Exiting 'extract_date' (result=10.03.2021)


dataset/png/A4dzLRpZW9ThwkJ74lJYwy5qHoUYTkEGBoEDkEUW_12_sf_0.png 2409 3436 292
{'sf_no': '108', 'sf_date': '10.03.2021', 'buyer_inn': None, 'buyer_kpp': None, 'seller_inn': '6154134421', 'seller_kpp': '615401001'}
Приложение № 1 к постановлению Правительства Российской Федерации от 26 декабря 2011 г. № 1137
(в редакции постановления Правительства Российской Федерации от 19 августа 2017 г. № 981)
Счет-фактура № 108 от 10 марта 2021 г. ()
Исправление № — от —- (1а)
Продавец: Общество с ограниченной ответственностью "АРМТПРИБОР" (2)
Аврос: _347905, Ростовская обл, Таганрог г, Подгорная ул, дом 75 (28)
ИНН/КПП продавца: 6154134421/615401001 (26)
Грузоотправитель и его адрес: Г. С ООИОДАИОЕНИУТЕОНОИНИ ВОР ОУНЫС НОЕ с). -7 ЧОБОЦОООСКОСЯ РОЛАНИНОНТ-КОЛОЕНОЕННИНЦНЛЛОНОЕ ООООНННИООАНННООНОНСАОННИ
Грузополучатель и его адрес: _Общество с ограниченной ответственностью "Первый Метизный Завод", 347900, г. Таганрог, ул. Люксембург, 2403 @
К платежно-расчетному документу Ис ПРООНОЛОНИ ЦОС НННЫЕОНе ЕО

2021-11-16 12:37:03.935 | DEBUG    | extractor:head_roi:287 - table roi (90, 709, 3246, 919)
2021-11-16 12:37:05.054 | DEBUG    | extractor:sf_no_and_date:93 - Entering 'extract_sf_no' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, ' 87 ОТ __'), kwargs={})
2021-11-16 12:37:05.055 | DEBUG    | extractor:sf_no_and_date:93 - Exiting 'extract_sf_no' (result=87)
2021-11-16 12:37:05.056 | DEBUG    | extractor:sf_no_and_date:94 - Entering 'extract_date' (args=(<extractor.SfInfoExtractor object at 0x7ff1781f45c0>, '4 МАРТА 2021'), kwargs={})
2021-11-16 12:37:05.056 | DEBUG    | extractor:sf_no_and_date:94 - Exiting 'extract_date' (result=04.03.2021)


dataset/png/U8KxU32ggs4n7AadD8kVo7L1ReOh2ucrog9KPgvl_6_sf_0.png 2409 3436 292
{'sf_no': '87', 'sf_date': '04.03.2021', 'buyer_inn': '9201006697', 'buyer_kpp': '920101001', 'seller_inn': '6154149805', 'seller_kpp': '615401001'}
Счет-фактура № 87 от __4 марта 2021г (1) ВОДЕ, МНР аси) соот счные о осалото тет ем Дин
Исправление № -- от == - (1а)
Продавец: ООО "ТЗСК" (2)
Адрес: 347927, Ростовская обл, Таганрог г, Пархоменко ул, дом № 22-Г, корпус Ш/ПШ, квартира Склад 1 (2а)
ИНН/КПП продавца: 6154149805/615401001 ! 3 (26)
Грузоотправитель и его адрес: он же (3)
Грузополучатель и его адрес: ФГУП "САП",299057, Севастополь г, Летчиков ул, дом 2 (4)
К платежно-расчетному документу № 330 от 24.02.2021 г. (5)
Покупатель: ФГУП "САП" (6)
Адрес: 299057, Севастополь г, Летчиков ул, дом 2 (ба)
ИНН/КПП покупателя: 9201006697/920101001 (66)
Валюта: наименование, код Российский рубль, 643 (7)
Идентификатер государственного контракта, договора (соглашения) (при наличии): 1922187322241442209206408 я (8)


['1247',
 '1259',
 '',
 '9.ЭФЕС0АООАДААРЕННОСТАИННЕПРИОНЛЬСТАРООСНКНОЙФЕДЕРАЦИЯСТ19МАГУ2017Г.1991',
 '0311007КУ',
 '888',
 'ЦО4796',
 None,
 '108',
 '87']